In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
import imageio
import sqlite3
from flask import g, Flask, render_template
from werkzeug.wrappers import Request, Response
from flask import Flask
from werkzeug.serving import run_simple

In [2]:
con = sqlite3.connect('Clicks.db')

In [3]:
cur = con.cursor()

# Create table
try: 
    cur.execute('''CREATE TABLE Clicks
                   (painting_id, object_name, user_id, x, y)''')
except:
    pass

# Insert a row of data
for _ in range(100):
    painting_id = 123
    object_name = "dog"
    user_id = np.random.randint(0, 10000)
    x = np.random.randint(150, 300)
    y = np.random.randint(675, 850)
    
    cur.execute(f"INSERT INTO Clicks VALUES ('{painting_id}', '{object_name}','{user_id}', {x}, {y})")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [4]:
DATABASE = 'Clicks.db'

app = Flask(__name__)

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
    return db

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

In [5]:
@app.route('/')
def index():
    
    img = image.imread("example.png")

    
    cur = get_db().cursor()
    
    q = cur.execute("""SELECT x, y
                       FROM Clicks 
                       WHERE painting_id == '123'
                       AND object_name == 'dog'""")
    
    points = [(r[0], r[1]) for r in q]
    
    min_x, min_y, max_x, max_y = find_edges(normalize(points))

    v_lines = [min_x, max_x]
    h_lines = [min_y, max_y]
    box = (min_x, max_x, min_y, max_y)
        
    plt.figure(figsize = (16, 16))
    ax = plt.gca()
    plt.plot()

    for line in v_lines:
        plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                   ls='-', lw=1.5)

    for line in h_lines:
        plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                   ls='-', lw=1.5)

    plt.axis('off')
    plt.imshow(img)
    plt.savefig('static/images/123_dog.png')
    
    return render_template('test.html', name = 'dog', url ='static/images/123_dog.png')

In [6]:
def normalize(points):
    a = np.array(points)
    mean, stdev = np.mean(a, axis = 0), np.std(a, axis=0)
    
    # Normalize values, and see if they are more than a standard deviation 
    # apart from the mean
    outliers = ((np.abs(a[:,0] - mean[0]) > 0.25 * stdev[0])
               * (np.abs(a[:,1] - mean[1]) > 0.25 * stdev[1]))
        
    return a[~outliers]

In [7]:
def find_edges(points):
    """Find the edges of the user clicks to determine the boxes"""
    min_x = min(points, key = lambda x: x[0])[0]
    min_y = min(points, key = lambda x: x[1])[1]

    max_x = max(points, key = lambda x: x[0])[0]
    max_y = max(points, key = lambda x: x[1])[1]
    
    return min_x, min_y, max_x, max_y

In [8]:
def score(click, box, time):
    
    # max score is 1000
    score = 100
    
    # score goes down by 25 per second
    score -= time * 2.5
    
    # split click into x and y coordinates
    x, y = click
       
    # extract box edges
    x_min, x_max, y_min, y_max = box
    
    # find the biggest box size
    size = max(x_max - x_min, y_max - y_min)
   
    # If the click is within the box, give full points
    if (x_min < x < x_max) and (y_min < y < y_max):
        return score
    else:
        # distance to closest point on x-axis
        dx = max(min_x - x, 0, x - max_x)
        # distance to closest point on y-axis
        dy = max(min_y - y, 0, y - max_y)
        # pythagoras theorem
        distance = np.sqrt(dx**2 + dy**2)
        if distance > size:
            return 0
        else:
            return ((0.15 * size) / (distance ** 1.75 / 10)) * score

In [ ]:
run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jan/2022 03:07:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 03:07:04] "GET /static/images/123_dog.png HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 03:07:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 03:07:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 03:07:39] "GET /static/images/123_dog.png HTTP/1.1" 200 -
